In [1]:
#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
# Keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.optimizers import SGD, Adam, Adadelta, RMSprop
import keras.backend as K
# Train-Test
from sklearn.model_selection import train_test_split
# Scaling data
from sklearn.preprocessing import StandardScaler
# Classification Report
from sklearn.metrics import classification_report

In [5]:
from keras.utils.np_utils import to_categorical

In [6]:
df = pd.read_csv('data.csv')

In [7]:
#Removing Missing Values
na = pd.notnull(df["Position"])
df = df[na]

In [8]:
#Get the required features for our classification problem
df = df[["Position", 'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']]

In [9]:
#Categorizing positions
forward_player = ["ST", "LW", "RW", "LF", "RF", "RS","LS", "CF"]
midfielder_player = ["CM","RCM","LCM", "CDM","RDM","LDM", "CAM", "LAM", "RAM", "RM", "LM"]
defender_player = ["CB", "RCB", "LCB", "LWB", "RWB", "LB", "RB"]

In [10]:
df.loc[df["Position"] == "GK", "Position"] = 0

df.loc[df["Position"].isin(defender_player), "Position"] = 1

df.loc[df["Position"].isin(midfielder_player), "Position"] = 2

df.loc[df["Position"].isin(forward_player), "Position"] = 3

In [11]:
#Standard Scaling of features
x = df.drop("Position", axis = 1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = pd.DataFrame(sc.fit_transform(x))
y = df["Position"]

In [15]:
#Converting prediction to categorical
y_cat=to_categorical(y)

In [16]:
#Test Train Split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2)

In [17]:
#Build the model
model = Sequential()
model.add(Dense(60, input_shape = (33,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(4, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 60)                2040      
_________________________________________________________________
dense_7 (Dense)              (None, 15)                915       
_________________________________________________________________
dropout_2 (Dropout)          (None, 15)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 64        
Total params: 3,019
Trainable params: 3,019
Non-trainable params: 0
_________________________________________________________________
C:\Users\hemam\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [18]:
# Fit the model and run for 10 epochs
model.fit(x_train, y_train, verbose=1, epochs=10)

Epoch 1/10
454/454 [==============================] - 15s 2ms/step - loss: 0.4660 - accuracy: 0.8086
Epoch 2/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3330 - accuracy: 0.8620
Epoch 3/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3210 - accuracy: 0.8718
Epoch 4/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3115 - accuracy: 0.8740
Epoch 5/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3016 - accuracy: 0.8744
Epoch 6/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3051 - accuracy: 0.8726
Epoch 7/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3091 - accuracy: 0.8731
Epoch 8/10
454/454 [==============================] - 1s 2ms/step - loss: 0.3081 - accuracy: 0.8743
Epoch 9/10
454/454 [==============================] - 0s 1ms/step - loss: 0.3336 - accuracy: 0.8671
Epoch 10/10
454/454 [==============================] - 0s 1ms/step - loss: 0.3039 - accuracy: 0.873

In [20]:
#Confusion Matrix
y_pred_class=model.predict_classes(x_test)
from sklearn.metrics import confusion_matrix

y_pred = model.predict(x_test)
y_test_class = np.argmax(y_test, axis=1)
confusion_matrix(y_test_class, y_pred_class)

C:\Users\hemam\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[ 408,    0,    0,    0],
       [   0, 1000,  151,    0],
       [   1,   64, 1240,   45],
       [   1,    3,  196,  521]], dtype=int64)

In [21]:
#Classification Report
from sklearn.metrics import classification_report
print(classification_report(y_test_class, y_pred_class))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       408
           1       0.94      0.87      0.90      1151
           2       0.78      0.92      0.84      1350
           3       0.92      0.72      0.81       721

    accuracy                           0.87      3630
   macro avg       0.91      0.88      0.89      3630
weighted avg       0.88      0.87      0.87      3630

